In [2]:
import os
import os.path
import pickle
import re
import sys
import csv
import re
import pandas as pd
from func_timeout import func_timeout
from pandarallel import pandarallel

In [3]:
pandarallel.initialize()

INFO: Pandarallel will run on 96 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [4]:
type_list = ['int', 'char', 'float', 'double', 'bool', 'void', 'short', 'long', 'signed', 'struct']

valid_ext = [".c", ".cpp"]

re_removed = "(\'.*\')|(\".*\")|(/\*.*\*/)|(//.*)"

keyword_set = set(type_list + ['sizeof'])

def pickle_dump(root_path, data, file_name):
    os.chdir(root_path)
    fp = open(file_name, "w")
    pickle.dump(data, fp)
    fp.close()

def pickle_load(root_path, file_name):
    os.chdir(root_path)
    fp_case = open(file_name, "r")
    dict_case = pickle.load(fp_case)
    fp_case.close()
    return dict_case


def is_valid_name(name):
    if re.match("[a-zA-Z_][a-zA-Z0-9_]*", name) == None:
        return False
    if name in keyword_set:
        return False
    return True

def is_func(line):
#int, __int64, void, char*, char *, struct Node, long long int, (void *)
#int func(int a, int *b, (char *) c)
    line = line.strip()
    if len(line) < 2:
        return None
# Rule 1: assume the function name line must ends with ) or {;
#    if line[-1] != ')' and line[-1] != '{':
#        return None
# Rule 2: (*) must in
    if '(' not in line: #or ')' not in line:
        return None
# Rule 3: # stands for #include or other primitives; / start a comment
    if line[0] == '#' or line[0] == '/':
        return None
# Rule 4: ends with ;
    if line.endswith(';'):
        return None
    if line.startswith('static'):
        line = line[len('static'):]
# replace pointer * and & as space
    line = re.sub('\*', ' ', line)
    line = re.sub('\&', ' ', line)


# replace '(' as ' ('
    #re.sub('(', ' ( ', line)
    line = re.sub('\(', ' \( ', line)
    line_split = line.split()

    if len(line_split) < 2:
        return None

    bracket_num = 0
    for ch in line:
        if ch == '(':
            bracket_num += 1

    has_type = False
    for type_a in type_list:
        if type_a in line_split[0]:
            has_type = True
#    if has_type == False:
#        return None
#    print line
    if bracket_num == 1:
        for index in range(len(line_split)):
            if '(' in line_split[index]:
                return line_split[index - 1]
    else:
        line = re.sub('\(', ' ', line)
        line = re.sub('\)', ' ', line)
        line_split = line.split()
        index = 0
        for one in line_split:
            if is_valid_name(one):
                index += 1
                if index == 2:
                    return one
        return None

def get_line_type(line):
    line = line.strip()
    if line.startswith("/*"):
#        print line
        return "comment_paragraph"
    elif line.startswith("//"):
        return "comment_line"
    elif line.startswith("#"):
        return "macro"
    return "other"

In [5]:
def is_comment_end(line):
    #print line
    line = line.strip()
    if line.endswith('*/'):
        return True
    return False

def is_func_end(line, left_brack_num):
    line = line.strip()
    left_brack_num += line.count("{")
    if "}" in line:
        left_brack_num -= line.count("}")
        if left_brack_num == 0:
            return True
    return False

# def func_name_extract(file_path):

#     if not os.path.isfile(file_path):
#         return


#     file_list = []
#     with open(file_path, "r", encoding='UTF8') as fp:
#          for line in fp.readlines():
#                  file_list.append(line)

def func_name_extract(df,i):
    content=df.iloc[i].content
    file_list=content.splitlines()
    func_list = []

    i = -1
    while i < len(file_list) - 1:
        i += 1
        line = file_list[i]
        line_type = get_line_type(line)
        if line_type == "comment_line" or line_type == "macro":
            continue
        elif line_type == "comment_paragraph":
            while not is_comment_end(file_list[i]):
                i += 1
        else:
            line = re.sub(re_removed, "", line)
            if len(line) == 0:
                continue
            func_name = is_func(line)
            if func_name != None:
#                print i, func_name
                start_line = i
                left_brack_num = 0
                effective_line = 1
                while True and i < len(file_list):
#                    print i
#                    print line
                    line = (file_list[i]).strip()
                    line_type = get_line_type(line)
                    if line_type == "comment_line":
                        continue

#                    elif line_type == "comment_paragraph":
#                        continue
#                        while not is_comment_end(file_list[i]):
#                            i += 1
                    left_brack_num += line.count('{')
                    effective_line += 1
                    if "}" in line:
                        left_brack_num -= line.count("}")
                        if left_brack_num < 1:
                            break
#                    print left_brack_num
                    i += 1
                end_line = i
               # if func_name != None:
                func_list.append([func_name, start_line + 1, end_line + 1, end_line - start_line + 1])
#                print func_name
    return func_list

In [6]:
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [7]:
df=pd.read_json('/home/hc3040/crypto_code_detection/data/full_data.json')

In [24]:
n=len(df)
tot=0

In [10]:
df=df.reset_index()

In [11]:
df.columns=['rn', 'data_source', 'label', 'file_name', 'is_header', 'content']

In [12]:
n=len(df)
def extracted_names(row):
    print("{}/{}".format(str(row.rn),n))
    
    try: 
        doitReturnValue = func_timeout(20, func_name_extract , args=(df,row.rn))
        return doitReturnValue
    except :
        print('time_out')
        return []

In [13]:
df['parsed_functions']=df.parallel_apply(extracted_names,axis=1)

2352/10736
2464/10736
3808/10736
1232/10736
560/10736
112/10736
3809/10736
3810/10736
561/10736
113/10736
562/10736
114/10736
563/10736
115/10736
336/10736
1344/10736
0/10736
564/10736
116/10736
224/10736
2016/10736
117/10736
448/10736
784/10736
118/10736
565/10736
2128/10736
337/10736
119/10736
566/10736
1/10736
120/10736
338/10736
1345/10736
2240/10736
1120/10736
2576/10736
2017/10736
785/10736
1680/10736
1008/10736
3136/10736
224/10736
1346/10736
2912/10736
121/10736
2/10736
339/10736
567/10736
786/10736
1347/10736
896/10736
6272/10736
340/10736
3/10736
122/10736
568/10736
2129/10736
787/10736
225/10736
1904/10736
341/10736
1792/10736
123/10736
2130/10736
569/10736
342/10736
124/10736
4/10736
3360/10736
343/10736
788/10736
570/10736
2577/10736
449/10736
226/10736
5/10736
3248/10736
4256/10736
5040/10736
789/10736
125/10736
2241/10736
3137/10736
227/10736
571/10736
344/10736
450/10736
126/10736
6/10736
3472/10736
1568/10736
6273/10736
2800/10736
3696/10736
4032/10736
2242/10736
228/1

845/10736
320/10736
84/10736
846/10736
85/10736
321/10736
521/10736
847/10736
322/10736
690/10736
86/10736
848/10736
87/10736
849/10736
691/10736
437/10736
88/10736
438/10736
439/10736
440/10736
850/10736
441/10736
89/10736
522/10736
90/10736
199/10736
91/10736
200/10736
92/10736
201/10736
93/10736
202/10736
94/10736
323/10736
95/10736
523/10736
442/10736
203/10736
324/10736
96/10736
524/10736
204/10736
325/10736
525/10736
326/10736
205/10736
97/10736
327/10736
206/10736
98/10736
99/10736
328/10736
851/10736
100/10736
443/10736
329/10736
207/10736
101/10736
852/10736
330/10736
208/10736
102/10736
444/10736
331/10736
103/10736
526/10736
209/10736
692/10736
332/10736
104/10736
210/10736
693/10736
211/10736
445/10736
212/10736
446/10736
213/10736
333/10736
853/10736
105/10736
214/10736
106/10736
334/10736
447/10736
854/10736
107/10736
215/10736
335/10736
855/10736
108/10736
216/10736
217/10736
527/10736
218/10736
528/10736
219/10736
529/10736
109/10736
220/10736
110/10736
530/10736
221/10

5942/10736
time_out
9075/10736
7958/10736
8850/10736
4935/10736
7061/10736
4936/10736
8851/10736
7062/10736
4937/10736
time_out
8852/10736
7063/10736
7398/10736
7620/10736
7064/10736
time_out
7621/10736
time_out
7065/10736
2693/10736
time_out
3033/10736
7622/10736
7066/10736
4711/10736
2694/10736
time_out
6390/10736
time_out
2695/10736
time_out
8516/10736
5607/10736
time_out
6391/10736
6506/10736
3925/10736
time_out
9296/10736
6507/10736
3926/10736
2696/10736
8517/10736
time_out
6508/10736
8518/10736
2697/10736
9631/10736
3927/10736
6509/10736
9632/10736
time_out
9633/10736
time_out
10184/10736
10295/10736
9634/10736
time_out
10296/10736
time_out
10185/10736
9635/10736
5834/10736
8638/10736
9636/10736
10297/10736
time_out
8639/10736
time_out
time_out
time_out
8739/10736
time_out
9637/10736
8963/10736
10629/10736
8640/10736
6728/10736
9409/10736
9638/10736
6729/10736
8641/10736
10630/10736
9410/10736
time_out
8964/10736
6730/10736
10631/10736
7734/10736
9411/10736
time_out
time_out
6731

9415/10736
time_out
1469/10736
time_out
10636/10736
6734/10736
5616/10736
9416/10736
7521/10736
1470/10736
6735/10736
9417/10736
5617/10736
1471/10736
10637/10736
6736/10736
9418/10736
10638/10736
1472/10736
9419/10736
10639/10736
1473/10736
time_out
9420/10736
1474/10736
time_out
9528/10736
1475/10736
7746/10736
time_out
9529/10736
10086/10736
7747/10736
time_out
9970/10736
7748/10736
10087/10736
9971/10736
time_out
10088/10736
9530/10736
9972/10736
time_out
7187/10736
time_out
9531/10736
6968/10736
time_out
10089/10736
time_out
8425/10736
9973/10736
7188/10736
9753/10736
6969/10736
8074/10736
9532/10736
10090/10736
9974/10736
time_out
7189/10736
time_out
8075/10736
5272/10736
9533/10736
6853/10736
9975/10736
10091/10736
7190/10736
6970/10736
9754/10736
5273/10736
6854/10736
7191/10736
6971/10736
5274/10736
6855/10736
6856/10736
time_out
6857/10736
9197/10736
time_out
6858/10736
9198/10736
9859/10736
6859/10736
time_out
9199/10736
9308/10736
9860/10736
time_out
6860/10736
9200/10736
9

time_out
2363/10736
time_out
2487/10736
2364/10736
2034/10736
2365/10736
time_out
2147/10736
time_out
3827/10736
time_out
4268/10736
4269/10736
time_out
3153/10736
time_out
time_out
3705/10736
2588/10736
3706/10736
4270/10736
2589/10736
time_out
3707/10736
time_out
1585/10736
4271/10736
2819/10736
3708/10736
4272/10736
time_out
time_out
2258/10736
5499/10736
time_out
time_out
1813/10736
5052/10736
time_out
1814/10736
1032/10736
5053/10736
1815/10736
time_out
time_out
5054/10736
1033/10736
time_out
10533/10736
5168/10736
time_out
5055/10736
3488/10736
1034/10736
1702/10736
1035/10736
time_out
time_out
3374/10736
4047/10736
time_out
10534/10736
3375/10736
time_out
4048/10736
4490/10736
3376/10736
4049/10736
6295/10736
4491/10736
3377/10736
4492/10736
3378/10736
4493/10736
4494/10736
4495/10736
time_out
10535/10736
4828/10736
4496/10736
time_out
10536/10736
4829/10736
4608/10736
10537/10736
4497/10736
time_out
4609/10736
4498/10736
4382/10736
time_out
time_out
4499/10736
4383/10736
3261/1

...

time_out
time_out
10589/10736
2094/10736
time_out
10590/10736
2538/10736
1305/10736
2095/10736
time_out
2539/10736
1306/10736
2096/10736
3883/10736
10591/10736
1307/10736
2540/10736
2097/10736
time_out
10592/10736
1308/10736
3211/10736
2541/10736
2542/10736
2543/10736
time_out
3441/10736
2544/10736
2545/10736
3442/10736
time_out
time_out
2546/10736
time_out
4110/10736
4334/10736
time_out
time_out
3756/10736
2547/10736
time_out
2868/10736
10593/10736
time_out
1421/10736
4335/10736
6348/10736
3757/10736
2637/10736
2548/10736
10594/10736
2869/10736
6349/10736
3758/10736
time_out
2638/10736
time_out
2549/10736
3443/10736
4671/10736
2212/10736
2639/10736
10595/10736
4672/10736
4673/10736
time_out
time_out
time_out
1091/10736
4547/10736
1759/10736
time_out
4548/10736
1760/10736
5556/10736
time_out
1761/10736
4549/10736
3994/10736
time_out
time_out
3111/10736
5783/10736
3112/10736
time_out
3659/10736
time_out
time_out
6222/10736
3660/10736
5233/10736
time_out
5118/10736
5234/10736
time_out
52

9060/10736
9061/10736
9062/10736
8158/10736
8159/10736
9063/10736
time_out
9847/10736
8160/10736
time_out
8727/10736
8728/10736
8729/10736
8730/10736
8731/10736
8732/10736
8733/10736
8734/10736
8735/10736
time_out
10733/10736
10734/10736
time_out
7828/10736
time_out
9942/10736
9943/10736
time_out
2301/10736
time_out
2424/10736
time_out
time_out
3887/10736
1983/10736
3888/10736
time_out
2425/10736
1984/10736
1635/10736
1985/10736
1986/10736
time_out
time_out
2102/10736
time_out
time_out
time_out
2996/10736
3541/10736
4889/10736
1313/10736
2997/10736
3542/10736
4890/10736
1314/10736
4891/10736
1315/10736
4892/10736
1316/10736
4893/10736
time_out
1317/10736
6353/10736
time_out
3762/10736
6354/10736
time_out
time_out
3218/10736
2874/10736
time_out
time_out
2219/10736
3219/10736
2875/10736
3220/10736
time_out
time_out
2553/10736
3449/10736
time_out
time_out
2554/10736
3997/10736
10599/10736
3450/10736
time_out
2555/10736
time_out
time_out
1425/10736
time_out
1767/10736
2647/10736
time_out
4

7941/10736
time_out
7942/10736
8057/10736
7943/10736
8058/10736
7944/10736
7945/10736
7946/10736
7947/10736
7948/10736
7949/10736
7950/10736
7951/10736
time_out
9400/10736
9401/10736
9402/10736
9403/10736
time_out
8171/10736
8172/10736
8173/10736
8174/10736
8175/10736
time_out
7835/10736
time_out
9956/10736
9957/10736
9958/10736
time_out
time_out
2307/10736
1642/10736
2308/10736
time_out
2309/10736
2110/10736
time_out
2310/10736
2111/10736
2311/10736
time_out
4901/10736
2312/10736
time_out
1320/10736
4902/10736
time_out
2430/10736
2313/10736
1992/10736
1321/10736
time_out
4903/10736
time_out
1993/10736
4904/10736
3223/10736
3003/10736
time_out
time_out
3004/10736
2222/10736
3224/10736
time_out
3454/10736
time_out
time_out
time_out
3892/10736
3769/10736
time_out
3770/10736
4001/10736
time_out
time_out
time_out
3771/10736
time_out
4683/10736
6367/10736
4555/10736
2883/10736
4002/10736
time_out
3772/10736
10605/10736
1431/10736
2884/10736
time_out
4684/10736
3773/10736
3005/10736
1772/107

9507/10736
9508/10736
time_out
5705/10736
time_out
5934/10736
5935/10736
time_out
6607/10736
time_out
time_out
time_out
10397/10736
10398/10736
10399/10736
10400/10736
time_out
time_out
5371/10736
5372/10736
5373/10736
time_out
1997/10736
time_out
2321/10736
time_out
time_out
2118/10736
time_out
1438/10736
6374/10736
time_out
1439/10736
1658/10736
time_out
6375/10736
3462/10736
1659/10736
6376/10736
3463/10736
time_out
1660/10736
3011/10736
3464/10736
6377/10736
time_out
3012/10736
6378/10736
time_out
2442/10736
time_out
2233/10736
3013/10736
time_out
1327/10736
2896/10736
time_out
time_out
4008/10736
2897/10736
time_out
5253/10736
3905/10736
4009/10736
time_out
time_out
time_out
time_out
5577/10736
time_out
time_out
2660/10736
5136/10736
time_out
3232/10736
time_out
2898/10736
time_out
2566/10736
4911/10736
2661/10736
1783/10736
3233/10736
5137/10736
4700/10736
time_out
2899/10736
time_out
3560/10736
3234/10736
4564/10736
4141/10736
4701/10736
time_out
4565/10736
10615/10736
4142/1073

4024/10736
4025/10736
4026/10736
4027/10736
4028/10736
time_out
time_out
4029/10736
time_out
5149/10736
2911/10736
4030/10736
time_out
5150/10736
time_out
time_out
time_out
3787/10736
3689/10736
time_out
5032/10736
3246/10736
5151/10736
time_out
2673/10736
4815/10736
3690/10736
5033/10736
3247/10736
5034/10736
time_out
time_out
5035/10736
4475/10736
3918/10736
3691/10736
3919/10736
time_out
2796/10736
time_out
2797/10736
6157/10736
time_out
6158/10736
time_out
6261/10736
6262/10736
time_out
5816/10736
6263/10736
time_out
5592/10736
time_out
4252/10736
4253/10736
4254/10736
4255/10736
time_out
6043/10736
6044/10736
6045/10736
6046/10736
6047/10736
time_out
2005/10736
time_out
2332/10736
2333/10736
2334/10736
time_out
2127/10736
time_out
1675/10736
1676/10736
1677/10736
1678/10736
1679/10736
time_out
1336/10736
time_out
3572/10736
time_out
2457/10736
2458/10736
time_out
4576/10736
4577/10736
4578/10736
time_out
time_out
time_out
4476/10736
4477/10736
time_out
3788/10736
time_out
time_out

In [25]:
df.columns

Index(['rn', 'data_source', 'label', 'file_name', 'is_header', 'content',
       'parsed_functions'],
      dtype='object')

In [26]:
df_save=df[['data_source', 'label', 'file_name', 'content',
       'parsed_functions']]

In [22]:
df_save.to_csv('full_data_with_function_names.csv',index=False)

In [23]:
test=pd.read_csv('full_data_with_function_names.csv')

In [24]:
test

,data_source,label,file_name,content,parsed_functions
0,crypto-library,1,nettle/eddsa-hash.c,/* eddsa-hash.c\n\n Copyright (C) 2014 Niels...,"[['_eddsa_hash', 46, 52, 7]]"
1,crypto-library,1,nettle/twofishdata.c,/* twofishdata.c\n\n Generates the permutati...,"[['q0', 27, 64, 38], ['q1', 66, 103, 38], ['ma..."
2,crypto-library,1,nettle/umac32.c,/* umac32.c\n\n Copyright (C) 2013 Niels Möl...,"[['umac32_set_key', 45, 57, 13], ['umac32_set_..."
3,crypto-library,1,nettle/cfb.h,/* cfb.h\n\n Cipher feedback mode.\n\n Cop...,"[['cfb_encrypt', 52, 77, 26], ['ctx', 80, 119,..."
4,crypto-library,1,nettle/eddsa.h,/* eddsa.h\n\n Copyright (C) 2014 Niels Möll...,"[['ed25519_sha512_sign', 56, 68, 13]]"
...,...,...,...,...,...
10731,others,0,__math.h,/* Inline math functions for i387.\n Copyrig...,"[['\\', 42, 45, 4], ['\\', 47, 50, 4], ['\\', ..."
10732,others,0,__ocl_iterator.h,//\n// Copyright (c) 2015-2016 The Khronos Gro...,[]
10733,others,0,暴力搜索_二维数组中查找.hpp,/*\n¶þÎ¬Êý×éÖÐ²éÕÒ https://www.nowcoder.com/pr...,"[['Find', 20, 35, 16], ['solve', 38, 45, 8]]"
10734,others,0,源.cpp,#include <iostream>\n#include <math.h>\n#inclu...,[]
